# Estimates with tools producing QIR

Azure Quantum Resource Estimation is built upon [QIR](https://www.qir-alliance.org/), the forward-looking, fully interoperable specification for quantum programs.  In this notebook, we are showing how to use the `azure.quantum` Python package to directly submit QIR to the Resource Estimation target.  We are using [PyQIR](https://github.com/qir-alliance/pyqir) to generate QIR, however the example works with any other source of QIR as well. PyQIR can help you as a library to generate QIR from other quantum programming languages, and thereby enabling their execution on Azure Quantum Resource Estimator.

## Getting started

We import several Python classes and functions from `azure.quantum`, `qiskit`, and `pyqir`.  Note that we are not using Qiskit to build quantum circuits, however, we are leveraging `AzureQuantumJob` and `job_monitor` which are built on top of the Qiskit ecosystem.

In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider
from azure.quantum.qiskit.job import AzureQuantumJob

from qiskit.tools.monitor import job_monitor

# Support code to transition from pyqir-generator to pyqir package
try:
    from pyqir.generator import BasicQisBuilder, SimpleModule
except:
    from pyqir import BasicQisBuilder, SimpleModule

We connect to the Azure Quantum workspace.

In [ ]:
provider = AzureQuantumProvider (
    resource_id = "",
    location = ""
)

Next, let's implement a generic function that takes as input the Azure Quantum provider and the QIR bitcode of the quantum program.  It returns as result an Azure Quantum job.  Resource Estimation input parameters can be passed via keyword arguments to the function.

In [ ]:
def resource_estimation_job_from_qir(provider: AzureQuantumProvider, bitcode: bytes, **kwargs):
    """A generic function to create a resource estimation job from QIR bitcode"""

    # Find the Azure Quantum Resource Estimator target from the provider
    backend = provider.get_backend('microsoft.estimator')

    # You can provide a name for the job via keyword arguments; if not,
    # we'll use QIR job as a default name
    name = kwargs.pop("name", "QIR job")

    # We extract some job specific arguments from the backend's configuration
    config = backend.configuration()
    blob_name = config.azure["blob_name"]
    content_type = config.azure["content_type"]
    provider_id = config.azure["provider_id"]
    output_data_format = config.azure["output_data_format"]

    # Finally, we are creating the Azure Quantum JSON object and return it
    return AzureQuantumJob(
        backend=backend,
        target=backend.name(),
        name=name,
        input_data=bitcode,
        blob_name=blob_name,
        content_type=content_type,
        provider_id=provider_id,
        input_data_format="qir.v1",
        output_data_format=output_data_format,
        input_params = kwargs,
        metadata={}
    )

## Running a sample quantum program

Let's now create some QIR bitcode using PyQIR generator.  Here, we build a controlled S gate using 3 T gates and 2 CNOT gates.

In [ ]:
module = SimpleModule("Controlled S", num_qubits=2, num_results=0)
qis = BasicQisBuilder(module.builder)

[a, b] = module.qubits[0:2]
qis.t(a)
qis.t(b)
qis.cx(a, b)
qis.t_adj(b)
qis.cx(a, b)

Before we submit the QIR to the resource estimator, let's take a look at the QIR
output for this module.  We can use the `ir` function from PyQIR for that
purpose, that generates human-readable instructions.

In [ ]:
print(module.ir())

The QIR we submit to the function defined above must be passed in bitcode
format.  We obtain this format by calling `bitcode` instead of `ir` on the
module.  We can also pass resource estimation specific arguments, e.g., setting
the error rate to 0.5%.

In [ ]:
job = resource_estimation_job_from_qir(provider, module.bitcode(), errorBudget=0.005)
job_monitor(job)
result = job.result()

Finally, we print the resource estimation table.

In [ ]:
result